In [11]:
pip install requests pandas

Doing reverse Geocoding for extracting quarters (Ortsteil), in addition to Stadtteil. Google Maps APi does not deliver such info, thats why i used Nominatum (Open Street Map Api). It is important to use User-Agent Header, in order to implement delay between several requests, if you dont want to be blocked.

In [18]:
import requests
import pandas as pd
import time

# Load the existing CSV file
input_file = 'universities_Berlin.csv'
output_file = 'universities_Berlin_full.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Custom User-Agent header
headers = {
    'User-Agent': 'MyGeocodingApp/1.0 (your.email@example.com)'
}

# Function to reverse geocode coordinates using Nominatim API
def reverse_geocode(lat, lng):
    url = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lng}&format=json&addressdetails=1"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad status codes
        data = response.json()
        if 'address' in data:
            address = data['address']
            # Try to get the most specific locality level available
            if 'suburb' in address:
                return address['suburb']
            elif 'quarter' in address:
                return address['quarter']
            elif 'neighbourhood' in address:
                return address['neighbourhood']
            elif 'city_district' in address:
                return address['city_district']
            else:
                print(f"No specific locality found for lat={lat}, lng={lng}")
        else:
            print(f"No address details found for lat={lat}, lng={lng}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed for lat={lat}, lng={lng}: {e}")
    return None

# Add a new column for Orsteil (quarters)
df['Orsteil'] = df.apply(lambda row: reverse_geocode(row['latitude'], row['longitude']), axis=1)

# Save the updated DataFrame to a new CSV file
df.to_csv(output_file, index=False)

print(f"Updated file saved as {output_file}")

Updated file saved as universities_Berlin_full.csv


Renamed Neighbourhood to Stadteil and stripped AGS code to the last 2 digits for uniform apprearance of this column for future joining and avoding inconsistency issues.

In [17]:
import pandas as pd

# Load the existing CSV file
input_file = 'universities_Berlin_full.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Rename the 'neighbourhood' column to 'Stadtteil'
df.rename(columns={'neighborhood': 'Stadtteil'}, inplace=True)

# Strip the AGS code column to display only the last 2 digits
df['AGS_Code'] = df['AGS_Code'].astype(str).str[-2:]

# Save the updated DataFrame to the same CSV file
df.to_csv(input_file, index=False)

print(f"Updated file saved as {input_file}")

Updated file saved as universities_Berlin_full.csv
